In [1]:
import pandas as pd 
import numpy as np 
import scipy
# import xlrd 
import sklearn

from Gibbs_model_probit import Gibbs_sampling

from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
from utils import baseline_lr,baseline_esnet,baseline_justmean
from utils import baseline_LogitElsnet,baseline_justmode,baseline_random,baseline_LogitLR,baseline_RanForest
from sklearn.model_selection import KFold
from scipy.stats import binom 
from scipy.stats import norm
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from tqdm import trange
import time

In [19]:
np.random.seed(123)

# generate simu data
N_sample = 1000
N_feature_t = 6
N_feature_f = 44
noise_level = 0
N_feature = N_feature_t + N_feature_f

feature_t = np.random.normal(0, 1, size=(N_sample, N_feature_t)) # true feature (should be select)
feature_f = np.random.normal(1, 0.5, size=(N_sample, N_feature_f)) # false feaures

W_t = np.array([-3, 3, -2, 2, -1, 1])#np.random.normal(0, 1, size=(N_feature_t,1))

Y = np.matmul(feature_t, W_t)

Y = Y + noise_level*np.random.normal(0, 1, size=Y.shape)
Y = np.where(Y>0,1,0)

X = np.concatenate((feature_t,feature_f),1)
print(X.shape,Y.shape)


(1000, 50) (1000,)


258

In [20]:
# random split the group
K = 10 # nunber of group
group_ind = []

'''
for i in range(K):
    N_t = N_feature_t//K
    N_f = N_feature_f//K
    idx_t = [N_t*i +j for j in range(N_t)]
    idx_f = [N_feature_t + N_f*i +j for j in range(N_f)]
    group_ind.append(idx_t + idx_f)

# so in each group, first 10 idx are the ture features
'''
for i in range(K):
    if i < 3:
        N_t = 2 #N_feature_t//K
        N_f = 3 #N_feature_f//K
        idx_t = [N_t*i +j for j in range(N_t)]
        idx_f = [N_feature_t + N_f*i +j for j in range(N_f)]
        group_ind.append(idx_t + idx_f)
    else:
        N_f = 5
        idx_f = [ N_f*i +j for j in range(N_f)]
        group_ind.append(idx_f)

group_ind
# 0-5 : relevant features
# 6-40: unrelevant features

[[0, 1, 6, 7, 8],
 [2, 3, 9, 10, 11],
 [4, 5, 12, 13, 14],
 [15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24],
 [25, 26, 27, 28, 29],
 [30, 31, 32, 33, 34],
 [35, 36, 37, 38, 39],
 [40, 41, 42, 43, 44],
 [45, 46, 47, 48, 49]]

In [21]:
# re-arrange the features of X based on the group split order
X_new = np.concatenate([X[:,group_ind[i]] for i in range(K)],axis=1)

# add all-one column at the last 
bias_col = np.ones(N_sample).reshape((N_sample,1))
X_new = np.concatenate((X_new,bias_col),axis=1)

print(X_new.shape)

(1000, 51)


In [22]:
# split train & test
X_train, X_test, y_train, y_test = train_test_split(X_new, Y.squeeze(),test_size=0.3)
data_dict = {'X_tr':X_train, 'y_tr':y_train, 'X_test':X_test, 'y_test':y_test}                                                       

In [23]:
# init hyper-parameters
alpha = 0.5
beta = 0.5
r0 = 1e-3
r1 = 1.0
a0 = 1.0
b0 = 1.0
JITTER = 1e-3

INTERVAL = 100
VALITA_INTERVAL = 500
BURNING = 2000
MAX_NUMBER = 3000

hyper_paras = {'INTERVAL':INTERVAL, 'BURNING':BURNING,'MAX_NUMBER':MAX_NUMBER,'VALITA_INTERVAL':VALITA_INTERVAL,
'alpha':alpha, 'beta':beta,'r0':r0,'r1':r1,'JITTER':JITTER}

In [13]:
# init parameters with lr_result
def get_init_paras(w_lr):
    z_array_init = np.zeros(K) #np.random.binomial(size=K, n=1, p= alpha)
    s_list_init = [np.zeros(len(item)) for item in group_ind]#[np.random.binomial(size=len(item), n=1, p= beta) for item in group_ind]
    b_init = w_lr[-1]#np.random.normal(loc=0.0, scale=r1,size=None)
    # tau_init = 1.0#np.random.gamma(shape=alpha, scale=1.0/beta, size=None)

    W_init = []
    offset=0
    for i in range(K):
        # mask1 = 1-z_array_init[i] * s_list_init[i]
        # mask2 = z_array_init[i] * s_list_init[i]
        # spike = np.random.normal(loc=0.0, scale=r0,size=len(s_list_init[i]))
        # slab = np.random.normal(loc=0.0, scale=r1,size=len(s_list_init[i]))
        # W_group = spike * mask1 + slab * mask2

        
        group_len = len(s_list_init[i])
        W_group= w_lr[offset:offset+group_len]
        offset = offset + group_len
        W_init.append(W_group)

    init_paras = {'z':z_array_init, 's':s_list_init, 'b':b_init,  'W':W_init,'a0':a0,'b0':b0}
    return init_paras

In [24]:

N = 5
lr_acc = np.zeros(N)
rf_acc = np.zeros(N)
esnet_acc = np.zeros(N)
mode_acc = np.zeros(N)
random_acc = np.zeros(N)
ours_acc = np.zeros(N)


lr_auc = np.zeros(N)
rf_auc = np.zeros(N)
esnet_auc = np.zeros(N)
ours_auc = np.zeros(N)


for i in range(N):
    X_train, X_test, y_train, y_test = train_test_split(X_new, Y.squeeze(),test_size=0.3)

    data_dict = {'X_tr':X_train, 'y_tr':y_train, 'X_test':X_test, 'y_test':y_test}  
    dict_lr = baseline_LogitLR(data_dict)
    dict_els = baseline_LogitElsnet(data_dict)
    dict_rf = baseline_RanForest(data_dict)
    dict_mode = baseline_justmode(data_dict)
    dict_random = baseline_random(data_dict)

    model = Gibbs_sampling(data_dict,get_init_paras(dict_lr['clf'].coef_.squeeze()), hyper_paras)
    dict_ours = model.model_run()

    lr_acc[i] = dict_lr['acr']
    esnet_acc[i] = dict_els['acr']
    rf_acc[i] = dict_rf['acr']
    mode_acc[i] = dict_mode['acr']
    random_acc[i] = dict_random['acr']
    ours_acc[i] = dict_ours['acr']


    lr_auc[i] = dict_lr['auc']
    rf_auc[i] = dict_rf['auc']
    esnet_auc[i] = dict_els['auc']
    ours_auc[i] = dict_ours['auc']


print('\n\nours_acr_mean: %.4f,ours_acr_std: %.4f '%( ours_acc.mean(), ours_acc.std() ) )
print('lr_acr_mean: %.4f,lr_acr_std: %.4f '%(lr_acc.mean(),lr_acc.std() ) )
print('esnet_acr_mean: %.4f,esnet_acr_std: %.4f '%(esnet_acc.mean(),esnet_acc.std() ) )
print('rf_acr_mean: %.4f,rf_acr_mean: %.4f '%(rf_acc.mean(),rf_acc.std() ) )
print('just-mode_acr_mean: %.4f,mode_acr_std: %.4f '%(mode_acc.mean(),mode_acc.std() ) )
print('just-random_acr_mean: %.4f,just-random_acr_std: %.4f '%(random_acc.mean(),random_acc.std() ) )

print('ours_AUC_mean: %.4f,ours_AUC_std: %.4f '%(ours_auc.mean(),ours_auc.std() ) )
print('lr_AUC_mean: %.4f,lr_AUC_std: %.4f '%(lr_auc.mean(),lr_auc.std() ) )
print('esnet_AUC_mean: %.4f,esnet_AUC_mean: %.4f '%(esnet_auc.mean(),esnet_auc.std() ) )
print('rf_AUC_mean: %.4f,rf_AUC_std: %.4f '%(rf_auc.mean(),rf_auc.std() ) )


C:\Users\Tianfan Wu\Documents\Python Scripts\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
for LogitLR: accuracy is 0.986667, auc is 0.998755,  fpr is 0.013514
for LogitElsnet: accuracy is 0.986667, auc is 0.998800,  fpr is 0.013514
C:\Users\Tianfan Wu\Documents\Python Scripts\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
for RandomForest: accuracy is 0.750000, auc is 0.835237,  fpr is 0.162162
for justmode: accuracy is 0.506667, fpr is 1.000000
for random-guess: accuracy is 0.503333, fpr is 0.500000
  0%|          | 0/3000 [00:00<?, ?it/s]
 running test-auc = 0.98969
running train-auc = 0.99244

 16%|█▌        | 481/3000 [00:02<00:12, 196.48it/s]
 running test-auc = 0.99973
running train-auc = 0

In [31]:
dict_lr['clf'].coef_

array([[-3.35738211e+00,  3.29937659e+00,  1.35294667e-02,
         2.18516424e-01,  2.48616333e-01, -2.21618577e+00,
         1.97401538e+00, -1.71221473e-02, -1.22510143e-01,
        -5.49657909e-02, -1.23690301e+00,  1.05391641e+00,
         1.23759977e-01, -5.49860576e-02, -6.20363376e-02,
        -7.38344665e-02, -1.66216076e-02, -1.53572276e-01,
         2.00061718e-04,  2.14143419e-01,  1.39165756e-01,
         1.02176841e-01,  1.81614700e-01,  9.35021195e-02,
         1.18232271e-02, -1.31431475e-01, -2.28807181e-01,
         3.93066093e-02,  7.75254502e-02, -2.19501529e-03,
        -2.03904368e-01,  2.44134043e-01, -2.94695558e-01,
         1.97691597e-01, -7.81276427e-02, -5.30709677e-02,
         7.70586113e-03,  8.47172273e-02, -1.60885627e-01,
        -3.49428285e-01,  1.48112764e-01,  1.03512076e-01,
        -6.43640319e-02, -1.71003596e-01,  3.79105508e-01,
        -1.87397671e-01, -1.38588743e-02,  1.35046949e-02,
        -3.65503760e-01, -2.17860799e-02,  1.74588857e-0

In [28]:
model.W_mean

array([-3.90903706,  3.65560616, -0.        ,  0.        , -0.        ,
       -2.64597942,  2.51789901, -0.        ,  0.01747499,  0.        ,
       -1.29812849,  0.99660596,  0.        ,  0.42239975, -0.        ,
       -0.        , -0.        ,  0.        , -0.        ,  0.        ,
       -0.        , -0.        ,  0.        ,  0.        ,  0.        ,
       -0.        , -0.        ,  0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        ,  0.        , -0.        ,
        0.        ,  0.        , -0.        ,  0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.10566843])

In [10]:
model.s_mean

[array([1, 1, 0, 0, 0]),
 array([1, 1, 0, 0, 0]),
 array([1, 1, 0, 0, 0]),
 array([0, 0, 0, 0, 0]),
 array([0, 1, 0, 1, 0]),
 array([1, 1, 0, 1, 0]),
 array([0, 1, 1, 1, 1]),
 array([0, 1, 0, 1, 1]),
 array([1, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 1])]